## Import Library

In [ ]:
import numpy as np
import pandas as pdx
import matplotlib.pyplot as plt
import random

## City Class

In [ ]:
class City:
    def __init__(self, name, x, y):
        self.name = name
        self.x = x
        self.y = y

    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance

    def __repr__(self):
        return f"({self.name})"

## Ant Colony Optimization Function

In [ ]:
def antColonyOptimization(city, iteration, nAnts, rho, alpha, beta, initialPheromne):
  cityList = []
  for i in range(0, len(city)):
    cityList.append(City(name = city.iloc[i,0],x=city.iloc[i][1],y=city.iloc[i][2]))

  distances = np.zeros((len(cityList), len(cityList)))
  visibility = np.zeros((len(cityList), len(cityList)))
  # Inisialization Pheromne
  pheromne = initialPheromne * np.ones((len(cityList), len(cityList)))
  print("Initial Pheromne")
  print("--------------------------------")
  print("Initail City | Destination City | New Pheromne")
  for row in range(len(cityList)):
    for col in range (len(cityList)):
      distance = cityList[row].distance(cityList[col])

      distances[row, col] = distance
      visibility[row, col] = 1/distance  if distance != 0 else 0
      if col > row:
        print(f"{row + 1}             | {col + 1}               | {pheromne[row, col]:.4f}")
  print("------------------------------------------\n")

  # Initialization Routes
  routes = np.ones((nAnts, len(cityList)+1), dtype=int)

  bestRoute = None
  bestDistance = float('inf')
  bestDistances = []

  # ACO Iteration
  for idx in range(iteration):
    print(f"=========================== Iteration {idx+1} ============================")
    antAndDistanceStr = ""
    # Initialization the first City index
    initialCitiesIdx = np.random.permutation(len(cityList))
    totalDistance = np.zeros((nAnts, 1)) # initiate total distance
    for i in range(nAnts):
      distance = 0
      # Assign first city to each ants
      routes[i, 0] = initialCitiesIdx[i] + 1
      # Copy the visibility
      visibilityTemp = np.array(visibility)

      for j in range(len(cityList)-1):
        currentLocation = int(routes[i, j] -1)
        # Set City Visibility to zero
        visibilityTemp[:, currentLocation] = 0

        pFeature = np.power(pheromne[currentLocation, :], beta)
        vFeature = np.power(visibilityTemp[currentLocation, :], alpha)
        features = np.multiply(pFeature, vFeature)
        total = np.sum(features)
        probabilities = features/total

        print(f"Semut {i + 1}: {routes[i, :]}")
        print("---------------------")
        print("Kota  | Probabilitas |")
        for k in range(len(cityList)):
          print(f"{k + 1}     | {probabilities[k]:.4f}       |")
        print("---------------------")

        # Choose next city with highest probability
        nextCityIdx = np.argmax(probabilities)
        routes[i, j+1] = nextCityIdx + 1 # Add next city to route

        distance += distances[int(routes[i, j]) - 1, int(routes[i, j+1]) - 1]


      routes[i, -1] = routes[i, 0] # Back to first City
      print(f"Semut {i + 1}: {routes[i, :]}")

      # Calculate last city to first city
      distance += distances[int(routes[i, -2]) - 1, int(routes[i, -1]) - 1]
      totalDistance[i] = distance
      antAndDistanceStr += f"Semut {i+1}: {'-'.join(map(str, map(int, routes[i, :])))} | Jarak = {totalDistance[i, 0]:.4f}\n"

      print("\n====================\n")

    print(f"Iteration {idx} Result: ")
    print(antAndDistanceStr)


    # Search the best routes
    distanceMinIdx = np.argmin(totalDistance)
    distanceMin = totalDistance[distanceMinIdx]
    if distanceMin < bestDistance:
      bestDistance = distanceMin
      bestRoute = routes[distanceMinIdx, :]

    bestDistances.append(bestDistance)

    # Update pheromne
    pheromne = (1 - rho) * pheromne # Evaporation
    for i in range(nAnts):
      delta = 1 / totalDistance[i][0] # Delta Pheromne
      for j in range(len(cityList)):
        pheromne[int(routes[i, j]) - 1, int(routes[i, j+1]) - 1] += delta
        pheromne[int(routes[i, j + 1]) - 1, int(routes[i, j]) - 1] += delta

    print("Update Pheromne")
    print("--------------------------------")
    print("Initail City | Destination City | New Pheromne")
    for i in range(len(cityList)):
      for j in range(i+1, len(cityList)):
        pheromneValue = pheromne[i, j]
        print(f"{i + 1}             | {j + 1}               | {pheromneValue:.4f}")
    print("------------------------------------------\n")


  print(f"The best routes: {'-'.join(map(str, map(int, bestRoute)))} | Total Distance = {bestDistance[0]:.4f}")

  # Ploting ACO Result
  plt.figure(1)
  plt.plot(range(1, iteration + 1), bestDistances)
  plt.xlabel('Iteration')
  plt.ylabel('Distance')
  plt.title('Ant Colony Optimization Result')
  plt.savefig('aco_result.png')

## Main

In [ ]:
%%capture cap
DatasetPath = 'lin318.csv'

# ACO Parameters
Iteration = 10
nAnts = 10
rho = 0.5
alpha = 1
beta = 1
InitialPheromne = 10

city = pdx.read_csv(DatasetPath, header=None , sep=' ')

print("====================================== Ant Colony Optimization ======================================\n")
print(f"Iteration: {str(Iteration)}")
print(f"Ants: {str(nAnts)}")
print(f"Rho: {str(rho)}")
print(f"Alpha: {str(alpha)}")
print(f"Beta: {str(beta)}")
print(f"Initial Pheromne : {str(InitialPheromne)}")
print()
antColonyOptimization(city, Iteration, nAnts, rho, alpha, beta, InitialPheromne)

with open('aco_result.txt', 'w') as f:
  f.write(cap.stdout)